In [21]:
from materials import materials
import pandas as pd
import os
# from pipe import pipe

In [12]:
materials.

SyntaxError: invalid syntax (3064698542.py, line 1)

In [22]:
# Write passes code

def passes_code():
    pass

def get_material_properties():
    pass

In [40]:
import pandas as pd

df = pd.read_csv(r'pipe\pipe_sizes.csv')
df['Inside Diameter'] = df['Outside Diameter'] - 2 * df['Wall Thickness']
df

,NPS,Identification,Schedule,Outside Diameter,Wall Thickness,Weight,Inside Diameter
0,0.125,NaN,10,0.405,0.049,0.19,0.307
1,0.125,NaN,10S,0.405,0.049,0.19,0.307
2,0.125,NaN,30,0.405,0.057,0.21,0.291
3,0.125,STD,40,0.405,0.068,0.24,0.269
4,0.125,STD,40S,0.405,0.068,0.24,0.269
...,...,...,...,...,...,...,...
610,44.000,XS,NaN,44.000,0.500,232.51,43.000
611,46.000,STD,NaN,46.000,0.375,182.90,45.250
612,46.000,XS,NaN,46.000,0.500,243.20,45.000
613,48.000,STD,NaN,48.000,0.375,190.92,47.250


In [27]:
def search_by_inside_diameter(df,
                              inside_diameter,
                              standard_only=True,
                              number_of_entries=None,
                              b31_1=False,
                              b31_3=False,
                              section_vii=False,
                              design_pressure=0,
                              design_temperature=0,
                              material=None,
                              seamless=True):
    """
    Searches for pipe entries closest to the specified inside diameter,
    with optional code compliance using materials.py.
    """
    if inside_diameter is None:
        raise ValueError("You must provide a target inside diameter.")

    filtered_df = df.copy()

    # Filter standard pipes only 
    if standard_only:
        filtered_df['Standard'] = ~(filtered_df['Identification'].isna() & filtered_df['Schedule'].isna())
        filtered_df = filtered_df[filtered_df['Standard']].drop(columns='Standard')

    # Filter by design codes if requested
    if any([b31_1, b31_3, section_vii]):
        if not material or design_pressure == 0 or design_temperature == 0:
            raise ValueError("Material, design_pressure, and design_temperature are required for code filtering.")

        # Use check_code_compliance from cell 6
        filtered_df = filtered_df[filtered_df.apply(
            lambda row: check_code_compliance(row, material, design_pressure, design_temperature,
                                              b31_1, b31_3, section_vii, seamless), axis=1)]

    # Compute absolute difference from target inside diameter
    filtered_df['ID Diff'] = abs(filtered_df['Inside Diameter'] - inside_diameter)

    # Sort by how close the inside diameter is
    filtered_df = filtered_df.sort_values(by='ID Diff')

    # Limit number of results if specified
    if number_of_entries:
        filtered_df = filtered_df.head(number_of_entries)

    # Drop helper column and reset index
    filtered_df = filtered_df.drop(columns=['ID Diff'])

    return filtered_df.reset_index(drop=True)


In [ ]:
# Always input these variables
inside_diameter = 0 #Enter the desired inside diameter as a decimal

# Optional variables
standard_only = False #False will include entries without identification or schedule
number_of_entries = 7 #Enter an integer if you only want to see a certain number of results, default is to see all entries
#TODO update your function above to work with the materials.py and make this block work with section viii= True
## Filter by Codes based on pressure and temperature if desired
section_viii = False #Set to True to only return results that pass Section VIII Div 1. UG-27 for design pressure and temperature
design_pressure = 4 #psig, must be entered if code filter is set to True
design_temperature = 5 #F, must be entered if code filter is set to True
material = None #'304', '316', '106 Gr B' are the currently available materials, must be entered if code filter is set to True
seamless = True #Set to False to apply a 0.85 strength reduction factor for welded pipe

# Ensure df is defined by running the cell that loads the CSV (cell 3) before this cell.
def search_by_inside_diameter(df, inside_diameter,
                              standard_only=False,
                              number_of_entries=None,
                              section_viii=False,
                              design_pressure=None,
                              design_temperature=None,
                              material=None,
                              seamless=True):
    pass


In [33]:
import numpy as np

# Simplified material stress lookup table
MATERIALS_DATA = {
    '106 Gr B': {'max_stress': 20000},
    '304': {'max_stress': 18000},
    '316': {'max_stress': 17000},
}

def lookup_water_density(pressure, temperature):
    """
    Returns water density in lbm/ft³ based on pressure and temperature.
    Uses XSteam for accurate property lookup.
    """
    steam_table_imp = XSteam(XSteam.UNIT_SYSTEM_FLS)  # imperial units (psig, F)
    return steam_table_imp.rho_pt(pressure, temperature)

def check_code_compliance(row, material, pressure, temperature, b31_1, b31_3, section_vii, seamless):
    """
    Simplified compliance check using Barlow’s formula:
    P = (2 * S * t) / D
    """
    if material not in MATERIALS_DATA:
        return False

    S = MATERIALS_DATA[material]['max_stress']
    if not seamless:
        S *= 0.85

    D = row['Outside Diameter']
    t = row['Wall Thickness']
    
    P_allow = (2 * S * t) / D
    return pressure <= P_allow

def calculate_velocity(mass_flow_rate, density, area_in2):
    """
    Calculate fluid velocity in ft/s given:
    - mass_flow_rate in lbm/s
    - density in lbm/ft³
    - area in in²
    """
    volumetric_flow_ft3s = mass_flow_rate / density
    area_ft2 = area_in2 / 144
    return volumetric_flow_ft3s / area_ft2

def search_by_fluid_velocity(df,
                             fluid_pressure,
                             fluid_temperature,
                             mass_flow_rate,
                             target_velocity,
                             standard_only=True,
                             number_of_entries=None,
                             b31_1=False,
                             b31_3=False,
                             section_vii=False,
                             design_pressure=0,
                             design_temperature=0,
                             material=None,
                             seamless=True):
    # Validate required inputs
    if None in [fluid_pressure, fluid_temperature, mass_flow_rate, target_velocity]:
        raise ValueError("fluid_pressure, fluid_temperature, mass_flow_rate, and target_velocity must be specified.")

    # Step 1: Estimate fluid density
    density = lookup_water_density(fluid_pressure, fluid_temperature)

    # Step 2: Calculate required pipe flow area
    volumetric_flow = mass_flow_rate / density  # ft³/s
    required_area_ft2 = volumetric_flow / target_velocity
    required_area_in2 = required_area_ft2 * 144  # convert to in²

    # Step 3: Filter and rank based on closest area
    working_df = df.copy()
    working_df = working_df.assign(
        AreaDiff=np.abs(working_df['Area'] - required_area_in2)
    ).sort_values('AreaDiff')

    # Step 4: Apply standard-only filter
    if standard_only:
        working_df = working_df[
            working_df['Identification'].isnull() |
            (working_df['Identification'].str.upper() == 'STD')
        ]

    # Step 5: Code compliance filtering
    if any([b31_1, b31_3, section_vii]):
        if design_pressure == 0 or design_temperature == 0 or material is None:
            raise ValueError("Code filtering requires design_pressure, design_temperature, and material.")
        working_df = working_df[working_df.apply(
            lambda row: check_code_compliance(row, material, design_pressure, design_temperature,
                                              b31_1, b31_3, section_vii, seamless), axis=1)]

    # Step 6: Calculate velocity for each row
    working_df['Velocity (ft/s)'] = working_df['Area'].apply(
        lambda area: calculate_velocity(mass_flow_rate, density, area)
    )

    # Step 7: Limit results
    if number_of_entries is not None:
        working_df = working_df.head(number_of_entries)

    return working_df[[
        'NPS', 'Schedule', 'Identification', 'Inside Diameter',
        'Outside Diameter', 'Wall Thickness', 'Area', 'Velocity (ft/s)'
    ]]

In [14]:
import numpy as np

def lookup_water_density(pressure, temperature):
    """Simplified placeholder: returns water density in lbm/ft³."""
    return 62.4  

def calculate_velocity(mass_flow_rate, density, area_in2):
    """Calculate fluid velocity in ft/s."""
    volumetric_flow = mass_flow_rate / density  # ft³/s
    area_ft2 = area_in2 / 144  # convert of unit in² to ft²
    return volumetric_flow / area_ft2

def get_pipe_info(df, nps, schedule=None, identification=None,
                  fluid_pressure=None, fluid_temperature=None,
                  mass_flow_rate=None):
    """
    Returns details for a specific pipe size and optional fluid flow info.
    """
    # Validate input
    if nps is None or (schedule is None and identification is None):
        raise ValueError("Must specify both NPS and either Schedule or Identification")

    # Lookup pipe row
    if schedule is not None:
        row = df[(df['NPS'] == nps) & (df['Schedule'].astype(str) == str(schedule))]
    else:
        row = df[(df['NPS'] == nps) & (df['Identification'].astype(str).str.upper() == str(identification).upper())]

    if row.empty:
        raise ValueError("No matching pipe found in database.")

    row = row.iloc[0]

    # Extract pipe geometry
    od = row['Outside Diameter']
    wt = row['Wall Thickness']
    id_ = row['Inside Diameter']
    area = row['Area']

    result = {
        'NPS': nps,
        'Schedule': schedule,
        'Identification': identification,
        'Outside Diameter (in)': od,
        'Wall Thickness (in)': wt,
        'Inside Diameter (in)': id_,
        'Flow Area (in^2)': area,
    }

    # Calculate fluid-related info if inputs are given
    if None not in [fluid_pressure, fluid_temperature, mass_flow_rate]:
        density = lookup_water_density(fluid_pressure, fluid_temperature)
        velocity = calculate_velocity(mass_flow_rate, density, area)
        result['Water Density (lbm/ft³)'] = density
        result['Mass Flow Rate (lbm/s)'] = mass_flow_rate
        result['Fluid Velocity (ft/s)'] = velocity

    return result

